In [1]:
%%time
import tensorflow as tf
import numpy as np
import pickle
import os
import json
import sys
import re
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from textwrap import wrap
import utils
from rdflib import Graph
import copy
import cv2
from datetime import datetime

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(gpu_options=gpu_options,log_device_placement=True,allow_soft_placement=True)

working_path = '/home/bobby/aida_copy/AIDA/M18_copy/data/'
corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_eval_m18/LDC2019E42_AIDA_Phase_1_Evaluation_Source_Data_V1.0/'
# corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_isi_dryrun3/dryrun-updated_tmp/dryrun/'
#corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_isi_dryrun3/dryrun/'
#corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_isi_dryrun/dryrun/' # D2

# Checking change from AIF Data
img_path = corpus_path
kfrm_path = corpus_path + 'data/video_shot_boundaries/representative_frames'
parent_child_tab = corpus_path + 'docs/parent_children.sorted.tab'
kfrm_msb = corpus_path + 'docs/masterShotBoundary.msb'


#rpi mention results in AIF
p_f = 'PT002_r2' # E5
p_f_share = 'E'
p_f_rpi = 'PT002_r2'

#grounding path
grounding_dict_path = working_path + 'all_features/grounding_dict_'+p_f+'.pickle'
#Todo: changed foldername
RPI_AIF_path = working_path + 'rpi_ttl/RPI_TA1_'+p_f_rpi # 1/7th May
#'RPI_TA1_20190504' 'rpi_ttl/RPI_dryrun_'+p_f
ltf_path = corpus_path + 'data/ltf/ltf'
# ltf_path = corpus_path + 'data/ltf/ltf'


video_asr_path = working_path + 'raw_files/RPI_TA1_'+p_f_share+'_asr/ltf_asr'
video_map_path = working_path + 'raw_files/RPI_TA1_'+p_f_share+'_asr/map_asr'
print('Check Point: RPI path change',RPI_AIF_path,video_asr_path)


# Checking change from Alireza
det_results_path_img = working_path + 'objdet_results/'+p_f_share+'/det_results_merged_34a_jpg.pkl'
# det_results_path_img = working_path + 'objdet_results/det_results_merged_32_jpg.pkl'
det_results_path_kfrm = working_path + 'objdet_results/'+p_f_share+'/det_results_merged_34b_kf.pkl'
# det_results_path_kfrm = working_path + 'objdet_results/det_results_merged_33_kf.pkl'
print('Check Point: Alireza path change:','\n',det_results_path_img,'\n', det_results_path_kfrm,'\n')


def attn(e,v,e_bar,v_bar):
    ## Inputs: local and global cap and img features ##
    ## Output: Heatmap for each word, Global Heatmap, Attnded Vis features, Corr-vals
    #e: ?xTxD, v: ?xNxD, e_bar: ?xD, v_bar: ?xN2xD
    with tf.variable_scope('word_level_attn'):
        #Eq.8
        s = tf.einsum('bij,bkj->bik',e,v) #pair-wise ev^T: ?xTxN
        
        #s_bar = tf.nn.softmax(s, axis=1) #softmax on words

        #Eq.9
        #alpha = tf.nn.softmax(gamma_1*s_bar, axis=2) #softmax on regions
        alpha = s
        c = tf.einsum('bij,bjk->bik',alpha,v) #?xTxD attnded visual reps for each of T words
        #Eq.10
        c_norm = tf.nn.l2_normalize(c,axis=2)
        e_norm = tf.nn.l2_normalize(e,axis=2)
        R_i = tf.einsum('bik,bik->bi',c_norm,e_norm) #cosine for T (words,img_reps) for all pairs
        R = tf.log(tf.pow(tf.reduce_sum(tf.exp(gamma_2*R_i),axis=1),1/gamma_2)) #? corrs
        N0=int(np.sqrt(alpha.get_shape().as_list()[-1]))
        heatmap_w = tf.reshape(alpha,[tf.shape(alpha)[0],tf.shape(alpha)[1],N0,N0])
    
    with tf.variable_scope('sen_level_attn'):
        #Eq.8
        s_s = tf.einsum('bj,bkj->bk',e_bar,v_bar) #pair-wise e_bar*v_bar^T: ?xN2
        
        #Eq.9
        #alpha_s = tf.nn.softmax(gamma_1_s*s_s, axis=1) #softmax on regions
        alpha_s = s_s
        c_s = tf.einsum('bj,bjk->bk',alpha_s,v_bar) #?xD attnded visual reps for sen.
        #Eq.10
        c_s_norm = tf.nn.l2_normalize(c_s,axis=1)
        e_bar_norm = tf.nn.l2_normalize(e_bar,axis=1)
        R_s = tf.einsum('bk,bk->b',c_s_norm,e_bar_norm) #cosine for (sen,img_reps)
        N0_g=int(np.sqrt(alpha_s.get_shape().as_list()[-1]))
        heatmap_s = tf.reshape(alpha_s,[-1,N0_g,N0_g])
        
    return heatmap_w, heatmap_s, c_norm, c_s_norm, R_i, R, R_s
def top_dict(kv_dict, num = 2):
    k_list = list(kv_dict.keys())[:num]
    subdict =  {k: kv_dict[k] for k in k_list}
    print('\ndict len:', len(kv_dict))
    return subdict

Check Point: RPI path change /home/bobby/aida_copy/AIDA/M18_copy/data/rpi_ttl/RPI_TA1_PT002_r2 /home/bobby/aida_copy/AIDA/M18_copy/data/raw_files/RPI_TA1_E_asr/ltf_asr
Check Point: Alireza path change: 
 /home/bobby/aida_copy/AIDA/M18_copy/data/objdet_results/E/det_results_merged_34a_jpg.pkl 
 /home/bobby/aida_copy/AIDA/M18_copy/data/objdet_results/E/det_results_merged_34b_kf.pkl 

CPU times: user 4.02 s, sys: 1.16 s, total: 5.19 s
Wall time: 13.6 s


In [2]:
%%time
print('Gathering keyframes info...')
#preparing dicts
id2dir_dict_kfrm = utils.create_dict_kfrm(kfrm_path, kfrm_msb, video_asr_path, video_map_path)
id2time_dict_kfrm = utils.id2time(kfrm_msb)

print('Creating parent-child dictionaries...')
#generate parent-child dictionaries
parent_dict, child_dict = utils.create_dict(parent_child_tab)

# for i in child_dict.keys():
#     if 'mp4' in i:
#         print(i,child_dict[i])
# print(top_dict(child_dict,5))

print('Creating path dictionary...')
#generate global id2path dictionary
#ltf
path_dict = utils.create_path_dict(os.path.join(corpus_path,'data/ltf/ltf/'))
#jpg
path_dict.update(utils.create_path_dict(os.path.join(corpus_path,'data/jpg/jpg/')))
#mp4
path_dict.update(utils.create_path_dict_kfrm(id2dir_dict_kfrm))

#print('Loading AIF crawl dictionaries...')
#with open(working_path + 'tmp/entity2mention_dict_r1.pickle', 'rb') as f:
#    en2men = pickle.load(f)

#with open(working_pathpath_pref + 'tmp/id2mentions_dict_r1.pickle', 'rb') as f:
#    id2men = pickle.load(f)

#loading a list of pronouns
with open(working_path+'raw_files/pronouns.txt', 'r') as f:
    lines = f.readlines()
    lines = [line.strip('\n') for line in lines]
    pronouns = set(lines)
    
print('Loading object detection results...')
#loading object detection results
with open(det_results_path_img, 'rb') as f:
    dict_obj = pickle.load(f)
    
with open(det_results_path_kfrm, 'rb') as f:
    dict_obj.update(pickle.load(f))
print('Loading done.')

Gathering keyframes info...
Creating parent-child dictionaries...
Creating path dictionary...
Loading object detection results...
Loading done.
CPU times: user 1.34 s, sys: 408 ms, total: 1.75 s
Wall time: 3.89 s


In [3]:
%%time
print('Loading AIF crawl dictionaries...')
with open(working_path + 'tmp/entity2mention_dict_'+p_f+'.pickle', 'rb') as f:
    en2men = pickle.load(f)

with open(working_path + 'tmp/id2mentions_dict_'+p_f+'.pickle', 'rb') as f:
    id2men = pickle.load(f)

# Check Point: en2men and id2men maybe None
print('en2men len:', len(en2men.keys()))
print('id2men len:', len(id2men.keys()))
print(datetime.now())

Loading AIF crawl dictionaries...
en2men len: 142899
id2men len: 2046
2019-07-11 23:17:10.843827
CPU times: user 2.12 s, sys: 338 ms, total: 2.45 s
Wall time: 3.41 s


In [20]:
%%time 
# maybe changed: turtle_files comes from RPI, .ttl maybe changed 
# about 1 hour 15 min
print('Creating id-sentence-entity dictionary...')
print(datetime.now())
en2men = {}
turtle_files = os.listdir(RPI_AIF_path)
import ltf_util as rpi_ltf_util
ltf_util = rpi_ltf_util.LTF_util(ltf_path)
for i,file in enumerate(turtle_files):
    if ".ttl" not in file:
        continue
#     print(i,file) # HC000VULD.ttl
#     # Done: text mention filter checking
#     if (file not in ['HC000VULD.ttl','HC000VULD.ttl']):
#         continue
#     else:
#         print('file',file)
    turtle_path = os.path.join(RPI_AIF_path, file)
    #loading turtle content
    turtle_content = open(turtle_path).read()
    g = Graph().parse(data=turtle_content, format='n3')
    # Check Point: en2men maybe changed
    #find 'aida:Entity', then find 'aida:justifiedBy'
#     print('g',len(en2men.keys()))
    en2men.update(utils.get_entity2mention(g,ltf_util)) 
    sys.stdout.write('File {}/{} \r'.format(i+1,len(turtle_files)))                
    sys.stdout.flush()
    
## mention type: mention, nominal_mention, pronominal_mention, normalized_mention
## Named entity types: PER PRG GPE LOC ORG WEA VEH
## Filler types: BAL COM CRM LAW MON RES SID TTL VAL

#short_to_long_map = {
#    'BAL': 'Ballot',
#    'COM': 'Commodity',
#    'CRM': 'Crime',
#    'FAC': 'Facility',
#    'GPE': 'GeopoliticalEntity',
#    'LAW': 'Law',
#    'LOC': 'Location',
#    'MON': 'Money',
#    'ORG': 'Organization',
#    'PER': 'Person',
#    'RES': 'Result',
#    'SID': 'Sides',
#    'TTL': 'Title',
#    'VAL': 'Value',
#    'VEH': 'Vehicle',
#    'WEA': 'Weapon',
# }
# filter_out=['pronominal_mention','GeopoliticalEntity','Organization',
#             'Location','Money','NumericalValue','Time','URL','Sides',
#             'Sentence','Results','Law','Crime','Ballot','Age']
print('Creating dictionary and filtering out for grounding...')
filter_out=[ 'pronominal_mention', 'GPE', 'ORG',
       'LOC','MON', 'NumericalValue', 'VAL', 'Time', 'URL', 'COM', 'SID',
        'Sentence', 'RES', 'LAW', 'CRM', 'BAL', 'Age', 'TTL'] 

id2men = utils.create_entity_dict(en2men, path_dict, caption_alignment_path=[], filter_out=filter_out)
# uncomment
print('Processing graphs done.')
open('log_grounding_'+p_f+'.txt', 'w').write('Processing graphs done.\n')
with open(working_path + 'tmp/entity2mention_dict_'+p_f+'.pickle', 'wb') as f:
    pickle.dump(en2men,f, protocol=pickle.HIGHEST_PROTOCOL)

with open(working_path + '/tmp/id2mentions_dict_'+p_f+'.pickle', 'wb') as f:
    pickle.dump(id2men,f, protocol=pickle.HIGHEST_PROTOCOL)
print(datetime.now())


Creating id-sentence-entity dictionary...
2019-07-10 23:56:47.032572
Creating dictionary and filtering out for grounding...
Processing graphs done.
2019-07-11 00:24:00.030831
CPU times: user 26min 35s, sys: 19.7 s, total: 26min 55s
Wall time: 27min 12s


In [4]:
%%time
#loading grounding pretrained model
print('Loading grounding pretrained model...')
gamma_1 = 10.0
gamma_1_s = 10.0
gamma_2 = 5.0
gamma_3 = 10.0
model_path = working_path + 'models/model_ELMo_PNASNET_VOA_norm'
#model_path = '../model_CNN_avg'
sess, graph = utils.load_model(model_path,config)
input_img = graph.get_tensor_by_name("input_img:0")
text_batch = graph.get_tensor_by_name("text_input:0")
mode = graph.get_tensor_by_name("mode:0")
v = graph.get_tensor_by_name("image_local_features:0")
v_bar = graph.get_tensor_by_name("image_global_features:0")
w_embedding = graph.get_tensor_by_name("w_embedding:0")
sen_embedding = graph.get_tensor_by_name("sen_embedding:0")
heatmap_w, heatmap_s, c, c_s, R_i, R, R_s = attn(w_embedding,v,sen_embedding,v_bar)
print('Loading done.')
print(datetime.now())

W0711 23:17:10.896356 140076587128640 deprecation_wrapper.py:119] From /home/bobby/aida_copy/AIDA/M18_copy/code_coconut/utils.py:874: The name tf.InteractiveSession is deprecated. Please use tf.compat.v1.InteractiveSession instead.



Loading grounding pretrained model...
Model Loading...


W0711 23:17:13.951940 140076587128640 deprecation_wrapper.py:119] From /home/bobby/aida_copy/AIDA/M18_copy/code_coconut/utils.py:875: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W0711 23:17:13.979271 140076587128640 deprecation_wrapper.py:119] From /home/bobby/aida_copy/AIDA/M18_copy/code_coconut/utils.py:875: The name tf.tables_initializer is deprecated. Please use tf.compat.v1.tables_initializer instead.

W0711 23:17:14.059724 140076587128640 deprecation_wrapper.py:119] From /home/bobby/aida_copy/AIDA/M18_copy/code_coconut/utils.py:877: The name tf.train.import_meta_graph is deprecated. Please use tf.compat.v1.train.import_meta_graph instead.



pass


W0711 23:17:20.086181 140076587128640 deprecation.py:323] From /dvmm-filer2/users/bobby/anaconda/envs/aida/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


----Model Loaded-----
Loading done.
2019-07-11 23:17:37.603166
CPU times: user 8.01 s, sys: 3.97 s, total: 12 s
Wall time: 26.8 s


# Grounding

In [25]:
%%time 
# about 6-8 hours in total for Grounding
#grounding, entity level
print(datetime.now())
# Todo: adjust parameters
en_score_thr = .5 #.9
sen_score_thr = .6 #.6
suffix_tmp = '_' + p_f + '_5-6'

en_to_img_dict = {}
img_to_feat_dict = {}
img_cnt_dict = {}
for k,key in enumerate(id2men):
    sys.stdout.write('Key {}/{} \r'.format(k,len(id2men)))                
    sys.stdout.flush()
    open('log_grounding_'+p_f+'.txt', 'w').write('Key {}/{} \r'.format(k,len(id2men)))
    #get all sens of the given leaf doc, and all images of its root doc
    imgs,ids = utils.fetch_img(key, parent_dict, child_dict, path_dict, level = 'Parent')
    sens = list(id2men[key].keys())
    if len(sens)==0:
        continue
    #generate all img-sen pairs and their corresponding grounding params
    text_flag = len(imgs)==0
    img_batch, sen_batch, img_info_batch = utils.img_cap_batch_gen(imgs,sens,ids,key,path_dict,id2time_dict_kfrm)
    placeholders = [input_img, text_batch, mode]
    tensor_list = [w_embedding, c, heatmap_w, R_i, R, R_s]
    inputs = [img_batch,sen_batch]
    EN_embd, IMG_embd, EN_heat, EN_score, avg_EN_score, sen_score = utils.batch_split_run(sess,tensor_list,placeholders,inputs,text_flag,b_size_thr=30)
    for i,sen in enumerate(sen_batch):
        #if sen_score[i] < sen_score_thr:
        #    continue
        for entity in id2men[key][sen]:
            en_name = id2men[key][sen][entity]['name']
            if en_name.lower() in pronouns:
                continue
            for mention in id2men[key][sen][entity]['mentions']:
                men_dict = id2men[key][sen][entity]['mentions'][mention]
                if men_dict['idx']==[]:
                    continue
            
                #if np.mean(EN_score[i,men_dict['idx']]) < en_score_thr:
                #    continue
                men_name = men_dict['name']
                if men_name.lower() in pronouns:
                    continue
                en_type = id2men[key][sen][entity]['type_rdf']
                source_type = id2men[key][sen][entity]['source_type']
                language = id2men[key][sen][entity]['language']
                             
                #men_embd = np.average(EN_embd[i,men_dict['idx'],:], weights = EN_score[i,men_dict['idx']], axis=0)
                men_embd = np.mean(EN_embd[i,men_dict['idx'],:], axis=0) #for now, just averaging word embdngs
                orig_img_id = img_info_batch[i][0]
                
                if text_flag:
                    grnd = {}
                elif sen_score[i] < sen_score_thr or np.mean(EN_score[i,men_dict['idx']]) < en_score_thr:
                    grnd = {}
                else:
                    heatmap = np.average(EN_heat[i,men_dict['idx'],:], weights = EN_score[i,men_dict['idx']], axis=0)
                    img_embd = np.average(IMG_embd[i,men_dict['idx'],:], weights = EN_score[i,men_dict['idx']], axis=0)
                    orig_img_shape = img_info_batch[i][1]
                    bbox_dict = utils.heat2bbox(heatmap,orig_img_shape)
                    bbox, bbox_norm, score = utils.filter_bbox(bbox_dict=bbox_dict, order='xyxy')
                    grnd = {'bbox': bbox, 'bbox_norm': bbox_norm, 'bbox_score': score,
                            'heatmap': heatmap, 'sen-img-score': sen_score[i],
                            'men-img-score': EN_score[i,men_dict['idx']],
                            'grounding_features': img_embd}
            
                men_dict = {mention: {'grounding': {orig_img_id: grnd},
                                      'textual_features': men_embd,
                                      'name': men_name,
                                      'sentence': sen}}
                en_dict = {'textual_features': np.zeros((men_embd.shape),dtype='float32'),
                           'name': en_name,
                           'type_rdf': en_type,
                           'mentions': men_dict,
                           'source_type': source_type,
                           'language': language}
                if entity not in en_to_img_dict:
                    en_to_img_dict[entity] = en_dict
                elif mention not in en_to_img_dict[entity]['mentions']:
                    en_to_img_dict[entity]['mentions'].update(men_dict)
                elif orig_img_id not in en_to_img_dict[entity]['mentions'][mention]['grounding']:
                    en_to_img_dict[entity]['mentions'][mention]['grounding'].update(men_dict[mention]['grounding'])
sess.close()

print('Grounding done.\nAveraging textual features to shape entity features...')
#averaging mention features to shape entity features
for entity in en_to_img_dict:
    for mention in en_to_img_dict[entity]['mentions']:
        en_to_img_dict[entity]['textual_features'] += en_to_img_dict[entity]['mentions'][mention]['textual_features']
    en_to_img_dict[entity]['textual_features'] /= len(en_to_img_dict[entity]['mentions'])

print('Averaging done.\nClustering mention bboxes to shape entity bbox...')
#clustering mention bboxes for each entity
for k,entity in enumerate(en_to_img_dict):
    men_dict = copy.deepcopy(en_to_img_dict[entity]['mentions'])
    # Done for USC: print dict of en_to_img_dict, and output
    en_to_img_dict[entity]['grounding'] = utils.men2en_grnd(men_dict,dict_obj)
    sys.stdout.write('Key {}/{} \r'.format(k,len(en_to_img_dict)))                
    sys.stdout.flush() 
#Todo alter filename
with open(grounding_dict_path + suffix_tmp + '.tmp', 'wb') as f:
    pickle.dump(en_to_img_dict,f, protocol=pickle.HIGHEST_PROTOCOL)
print('Clustering done.', 'grounding_dict.tmp len:',len(en_to_img_dict))
open('log_grounding_'+p_f+'.txt', 'w').write('\nGrounding Stage done.')
print(datetime.now())

2019-07-11 00:48:27.735141
Grounding done.
Averaging textual features to shape entity features...
Averaging done.
Clustering mention bboxes to shape entity bbox...
Clustering done. grounding_dict.tmp len: 42905
2019-07-11 15:35:37.795029
CPU times: user 10d 9h 33min 47s, sys: 1d 44min 25s, total: 11d 10h 18min 13s
Wall time: 14h 47min 10s


# Instance Features

In [26]:
%%time
#loading instance matching pretrained model
model_path = working_path + 'models/model_universal_no_recons_ins_only'
sess_ins, graph_ins = utils.load_model(model_path,config)
input_img_ins = graph_ins.get_tensor_by_name("input_img:0")
mode_ins = graph_ins.get_tensor_by_name("mode:0")
img_vec_ins = graph_ins.get_tensor_by_name("img_vec:0")

Model Loading...
pass


/dvmm-filer2/users/bobby/anaconda/envs/aida/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


----Model Loaded-----
CPU times: user 8.6 s, sys: 6.14 s, total: 14.7 s
Wall time: 9.52 s


In [27]:
def batch_of_bbox(img, bbox_norm):
    img_batch = np.empty((len(bbox_norm),224,224,3), dtype='float32')
    for i,bbox in enumerate(bbox_norm):
        roi = utils.crop_resize_im(img, bbox, (224,224), order='xyxy')
        img_batch[i,:,:,:] = roi
    return img_batch

In [28]:
%%time
# about 2 hours
suffix = suffix_tmp #'_8-6'
with open(grounding_dict_path + suffix_tmp +'.tmp', 'rb') as f:
    en_to_img_dict = pickle.load(f)
print(len(en_to_img_dict))

missed_children_ins = []
maxbatch = 0
for i,en in enumerate(en_to_img_dict):
    try:
#         if i in [11996,12085]:
#             continue
        for img_id in en_to_img_dict[en]['grounding']:
            imgs,_ = utils.fetch_img(img_id, parent_dict, child_dict, path_dict, level = 'Child')
            if len(imgs)==0:
                missed_children_ins.append(key)
                continue
            bbox_norm = en_to_img_dict[en]['grounding'][img_id]['bbox_norm']
            img_batch = batch_of_bbox(imgs[0], bbox_norm)
            feed_dict = {input_img_ins: img_batch, mode_ins: 'test'}
            img_vec_pred = sess_ins.run([img_vec_ins], feed_dict)[0]
            en_to_img_dict[en]['grounding'][img_id]['instance_features']=[]
            for j in range(len(bbox_norm)):
                en_to_img_dict[en]['grounding'][img_id]['instance_features'].append(img_vec_pred[j,:])
        sys.stderr.write("Stored for image {} / {} \r".format(i, len(en_to_img_dict)))
        sys.stdout.flush() 
    except ValueError:
        print("Oops!",i,ValueError) 
sess_ins.close()

with open(grounding_dict_path + suffix, 'wb') as f:
    pickle.dump(en_to_img_dict,f, protocol=pickle.HIGHEST_PROTOCOL)
print(datetime.now())

42905


2019-07-11 15:43:49.105216
CPU times: user 1h 30min 8s, sys: 6min 35s, total: 1h 36min 44s
Wall time: 8min 1s


In [29]:
sess_ins.close()
tf.__version__

'1.14.0-rc1'

# Saving

In [30]:
%%time
# # Todo: uncomment
print(datetime.now())
with open(grounding_dict_path + suffix, 'rb') as f:
    en_to_img_dict = pickle.load(f)
    
with open(grounding_dict_path, 'wb') as f: #/home/bobby/aida_copy/AIDA/M18_copy/data/all_features
    pickle.dump(en_to_img_dict,f, protocol=pickle.HIGHEST_PROTOCOL)
print('All done.')
open('log_grounding_'+p_f+'.txt', 'w').write('\nAll done.')

2019-07-11 15:43:49.218352
All done.
CPU times: user 5.26 s, sys: 2.24 s, total: 7.51 s
Wall time: 9.2 s


In [11]:
# suffix = '_1-6' #suffix #'_75-6'

# Visualization

In [31]:
%%time
# test for mention level
with open(grounding_dict_path + suffix, 'rb') as f:
    en_to_img_dict = pickle.load(f)
# todo testing
g_suffix = suffix #'_2+5-6' #suffix #'_75-6'
g_threshold_low = 0 #low bound: filter the entities with the en_score < threshold  
g_threshold_high = 1 #high bound: filter the entities with the en_score < threshold  

grounded_stat = {}

def add_hist_stat(stat_dict, s1, s2):
    key = str(round(s1,1)) + ' ' + str(round(s2,1))
    if key not in stat_dict.keys():
        stat_dict[key] = 1
    else:
        stat_dict[key] += 1
    return stat_dict

#mention level
pdf = matplotlib.backends.backend_pdf.PdfPages(working_path+'Grounding_all'+g_suffix+'.pdf') #("G_RU_UK.pdf")
for k,en_id in enumerate(en_to_img_dict):
    sys.stderr.write("Stored for image {} / {} \r".format(k, len(en_to_img_dict)))
    sys.stdout.flush()
    for men_id in en_to_img_dict[en_id]['mentions']:
        en_name = en_to_img_dict[en_id]['mentions'][men_id]['name']
        sen = en_to_img_dict[en_id]['mentions'][men_id]['sentence']
        lan = en_to_img_dict[en_id]['language']
        # Todo remove English
#         if lan=='English':
#             continue
        for img_id in en_to_img_dict[en_id]['mentions'][men_id]['grounding']:
            if len(en_to_img_dict[en_id]['mentions'][men_id]['grounding'][img_id])==0:
                continue
            bboxes = en_to_img_dict[en_id]['mentions'][men_id]['grounding'][img_id]['bbox_norm']
            heatmap = en_to_img_dict[en_id]['mentions'][men_id]['grounding'][img_id]['heatmap']
            sen_score = en_to_img_dict[en_id]['mentions'][men_id]['grounding'][img_id]['sen-img-score']
            en_score = np.mean(en_to_img_dict[en_id]['mentions'][men_id]['grounding'][img_id]['men-img-score'])
            
            if (en_score<=g_threshold_low or en_score>=g_threshold_high):
                continue
            #add
            grounded_stat = add_hist_stat(grounded_stat, en_score, sen_score)
            if img_id.find('mp4')>-1:
                vname_arr = img_id.split('_')
                vid = vname_arr[0] + '.' + '.'.join(vname_arr[1].split('.')[1:])             
                fid = ' '.join(child_dict[vid])
                title= fid + ' ' + img_id + ' '+ lan+'-Keyframe'
#                 print(title)
            else:
                fid = ' '.join(child_dict[img_id])
                title= fid + ' ' + img_id + ' '+ lan+'-JPG'
            #img,_ = utils.fetch_img(img_id, parent_dict, child_dict, path_dict, level = 'Child')
            title_str = title+', ' + 'Sen-Img Score: %.2f'%sen_score
            title_str = title_str + '\n sentence:' + sen
            title = "\n".join(wrap(title_str, 120))
            #fig = utils.img_heat_bbox_disp(cv2.resize(img[0],(299,299)), heatmap, title=title, en_name=en_name+', '+'%.2f'%en_score, bboxes=bboxes, order='xyxy')
            #pdf.savefig(fig)
            if lan!='English':
                en_name = ' '.join(en_name.split()[:2])
            img,_ = utils.fetch_img(img_id, parent_dict, child_dict, path_dict, level = 'Child')
            fig = utils.img_heat_bbox_disp(cv2.resize(img[0],(299,299)), heatmap, title=title, en_name=en_name+', '+'%.2f'%en_score, bboxes=bboxes, order='xyxy', show=False)
            pdf.savefig(fig)
pdf.close()
print(datetime.now())
print(sum(grounded_stat.values()))

error: OpenCV(4.1.0) /io/opencv/modules/core/src/alloc.cpp:55: error: (-4:Insufficient memory) Failed to allocate 814080 bytes in function 'OutOfMemoryError'


In [ ]:
%%time
#entity level
import cv2

# Load
with open(grounding_dict_path + '_.5', 'rb') as f:
    en_to_img_dict = pickle.load(f)
print(len(en_to_img_dict))


pdf = matplotlib.backends.backend_pdf.PdfPages(working_path+"Grounding_visualization.pdf")
for k,en_id in enumerate(en_to_img_dict):
#     if(k>5000):
#         break
    sys.stderr.write("Stored for image {} / {} \r".format(k, len(en_to_img_dict)))
    
#     for id in en_to_img_dict[en_id]['mentions']:
#         e = en_to_img_dict[en_id]['mentions'][id]
#         print(en_to_img_dict[en_id])
#         print( e['name'],'|','sen:',e['sentence'])
    
    for img_id in en_to_img_dict[en_id]['grounding']:
        en_name = en_to_img_dict[en_id]['name']
        bboxes = en_to_img_dict[en_id]['grounding'][img_id]['bbox_norm']
        heatmap = en_to_img_dict[en_id]['grounding'][img_id]['heatmap'][0]
        en_score = np.mean(en_to_img_dict[en_id]['grounding'][img_id]['men-img-score'])
        if img_id.find('mp4')>-1:
            title='Keyframe'
        else:
            title='JPG'
        img,_ = utils.fetch_img(img_id, parent_dict, child_dict, path_dict, level = 'Child')
        fig = utils.img_heat_bbox_disp(cv2.resize(img[0],(299,299)), heatmap, title=title, en_name=en_name+', '+'%.2f'%en_score, bboxes=bboxes, order='xyxy', show=True)
        pdf.savefig(fig)
pdf.close()

In [ ]:
%%time
#mention level
pdf = matplotlib.backends.backend_pdf.PdfPages("G_RU_UK.pdf")
for k,en_id in enumerate(en_to_img_dict):
    sys.stderr.write("Stored for image {} / {} \r".format(k, len(en_to_img_dict)))
    sys.stdout.flush()
    for men_id in en_to_img_dict[en_id]['mentions']:
        en_name = en_to_img_dict[en_id]['mentions'][men_id]['name']
        sen = en_to_img_dict[en_id]['mentions'][men_id]['sentence']
        lan = en_to_img_dict[en_id]['language']
        if lan=='English':
            continue
        for img_id in en_to_img_dict[en_id]['mentions'][men_id]['grounding']:
            if len(en_to_img_dict[en_id]['mentions'][men_id]['grounding'][img_id])==0:
                continue
            bboxes = en_to_img_dict[en_id]['mentions'][men_id]['grounding'][img_id]['bbox_norm']
            heatmap = en_to_img_dict[en_id]['mentions'][men_id]['grounding'][img_id]['heatmap']
            sen_score = en_to_img_dict[en_id]['mentions'][men_id]['grounding'][img_id]['sen-img-score']
            en_score = np.mean(en_to_img_dict[en_id]['mentions'][men_id]['grounding'][img_id]['men-img-score'])
            if img_id.find('mp4')>-1:
                title=lan+'-Keyframe'
            else:
                title=lan+'-JPG'
            #img,_ = utils.fetch_img(img_id, parent_dict, child_dict, path_dict, level = 'Child')
            #title_str = sen+',  Sen-Img Score: %.2f'%sen_score
            #title = "\n".join(wrap(title_str, 120))
            #fig = utils.img_heat_bbox_disp(cv2.resize(img[0],(299,299)), heatmap, title=title, en_name=en_name+', '+'%.2f'%en_score, bboxes=bboxes, order='xyxy')
            #pdf.savefig(fig)
            if lan!='English':
                en_name = ' '.join(en_name.split()[:2])
            img,_ = utils.fetch_img(img_id, parent_dict, child_dict, path_dict, level = 'Child')
            fig = utils.img_heat_bbox_disp(cv2.resize(img[0],(299,299)), heatmap, title=title, en_name=en_name+', '+'%.2f'%en_score, bboxes=bboxes, order='xyxy', show=False)
            pdf.savefig(fig)
pdf.close()

# Testing

In [8]:
suffix_tmp = '_' + p_f + '_8-6'
suffix = suffix_tmp #'_8-6'
with open(grounding_dict_path + suffix, 'rb') as f:
    en_to_img_dict = pickle.load(f)

In [13]:
#all grounded entities
s=0
for en in en_to_img_dict:
    for img_id in en_to_img_dict[en]['grounding']:
        if len(en_to_img_dict[en]['grounding'][img_id])>0:
            s+=1
#             print(en_to_img_dict[en]['type_rdf'])
print('all entities', len(en_to_img_dict))
print('grounded entities',s,'\n')



#all grounded mentions
e=0
m=0
s=0

for k,en_id in enumerate(en_to_img_dict):
    e+=1
    for men_id in en_to_img_dict[en_id]['mentions']:
        m+=1
        for img_id in en_to_img_dict[en_id]['mentions'][men_id]['grounding']:
            if len(en_to_img_dict[en_id]['mentions'][men_id]['grounding'][img_id])>0:
                s+=1
#             print(en_to_img_dict[en]['type_rdf'])
print('all entities', len(en_to_img_dict))
print('all mentiones',m)
print('grounded mentiones',s,'\n')

all entities 42604
grounded entities 1569 

all entities 42604
all mentiones 80510
grounded mentiones 5655 



In [ ]:
                
def check_grounding_dict(grounding_dict, child_dict):
    # test
#     key = ''
#     print('arrrr', grounding_dict[key]['grounding']['system'])
    
    # Check Point: grounding_dict printing
    print('cu_grounded')
    for en in grounding_dict.keys():
        for img_id in grounding_dict[en]['grounding']:
            if img_id  =='system':
                continue
            n_b = len(grounding_dict[en]['grounding'][img_id]['bbox'])
            if n_b>0:
                img_dict = grounding_dict[en]['grounding'][img_id]
                print(en)
                if img_id.find('mp4')>-1:
                    vname_arr = img_id.split('_')
                    img_id = vname_arr[0] + '.' + '.'.join(vname_arr[1].split('.')[1:])  
            
                
                print(child_dict[img_id], img_id, grounding_dict[en]['grounding'][img_id])
                print(sum(sum(img_dict['grounding_features'])),sum(sum(img_dict['instance_features'])))
                print('\n')  
check_grounding_dict(grounding_dict, child_dict)


# fid = ' '.join(child_dict[img_id])


#         if img_id.find('mp4')>-1:
                 
#                 fid = ' '.join(child_dict[img_id])
#                 title= fid + ' ' + img_id + ' '+ lan+'-Keyframe'
# #                 print(title)
#             else:
#                 fid = ' '.join(child_dict[img_id])
#                 title= fid + ' ' + img_id + ' '+ lan+'-JPG'

In [ ]:
id2men

In [ ]:
for en in en_to_img_dict:
    print(en_to_img_dict[en]['name'])

In [ ]:
# Check Point: grounding_dict printing
sub_en2img_dict = {}
for en in en_to_img_dict:
    for img_id in en_to_img_dict[en]['grounding']:
        n_b = len(en_to_img_dict[en]['grounding'][img_id]['bbox'])
        if n_b>0:
            print(en)
            print(img_id, child_dict[img_id])
            print('\n')
            sub_en2img_dict[en] = img_id

# cu_grndg_ent_pref = 'http://www.columbia.edu/AIDA/DVMM/Entities/GroundingBox/'+merge_version+'/'
# cu_grndg_type_pref = 'http://www.columbia.edu/AIDA/DVMM/TypeAssertions/GroundingBox/'+merge_version+'/'
# cu_grndg_clstr_img_pref = 'http://www.columbia.edu/AIDA/DVMM/Clusters/BoxOverlap/'+merge_version+'/'
# cu_grndg_clstr_txt_pref = 'http://www.columbia.edu/AIDA/DVMM/Clusters/Grounding/'+merge_version+'/'
        

In [ ]:
# Check Point: printing
# en='http://www.isi.edu/gaia/entities/c5137f4c-0d65-4d67-ad50-8338c610285f'
# gid = 'IC0011VED.jpg.ldcc'
# en_to_img_dict[en]['grounding'][gid]
def top_dict(kv_dict, num = 2):
    k_list = list(kv_dict.keys())[:num]
    subdict =  {k: kv_dict[k] for k in k_list}
#     print(subdict)
    return subdict
# top_dict(sub_en2img_dict)
print('sub_en2img_dict lens:',len(list(sub_en2img_dict.keys())))
sub_enlist = list(sub_en2img_dict.keys())[:3]
print(sub_enlist)
for en in sub_enlist:
    sub_imglist = list(en_to_img_dict[en]['grounding'].keys())[:1]
    print(sub_imglist)
    sub_imgdict =  {k: en_to_img_dict[en]['grounding'][k] for k in sub_imglist}
    print(sub_imgdict)
print(child_dict[img_id])


In [ ]:
%%time
train_writer = tf.summary.FileWriter('./logs', graph)

In [ ]:
%%time
!tensorboard --logdir ./logs --port=8993

In [ ]:
%%time
#checking whether english entities are subsets of all or not
# maybe changed
from rdflib import RDF, URIRef
# nist_ont_pref = '.../SM-KBP/2018/ontologies/InterchangeOntology#'
nist_ont_pref = 'https://tac.nist.gov/tracks/SM-KBP/2019/ontologies/InterchangeOntology#'
entity_ = URIRef(nist_ont_pref+'Entity')
RPI_AIF_path_en = working_path + 'rpi_ttl/ttl_en_r0'
RPI_AIF_path_all = working_path + 'rpi_ttl/ttl_all_r0'
turtle_files_en = os.listdir(RPI_AIF_path_en)
turtle_files_all = os.listdir(RPI_AIF_path_all)
print('totel len:',len(turtle_files_en))
for i,file in enumerate(turtle_files_en):
    print(file)
    if ".turtle" not in file:
        continue
    if i%1000 ==0:
        print('i = ',i)
    turtle_path_en = os.path.join(RPI_AIF_path_en, file)
    turtle_path_all = os.path.join(RPI_AIF_path_all, file)
    #loading turtle content
    turtle_content_en = open(turtle_path_en).read()
    turtle_content_all = open(turtle_path_all).read()
    g_en = Graph().parse(data=turtle_content_en, format='n3')
    g_all = Graph().parse(data=turtle_content_all, format='n3')
    entities_all = list(g_all.subjects(predicate=RDF.type,object=entity_))
    entities_en = list(g_en.subjects(predicate=RDF.type,object=entity_))
    for en in entities_en:
        if en not in entities_all:
            print(en)
    print(file)

In [ ]:
#old fetch img
def fetch_img(child_id, parent_dict, child_dict, path_dict, level='Child'):
    if child_id not in child_dict:
        return [],[]
    
    elif level=='Child':
        if child_id.find('jpg')!=-1:
            filename = path_dict[child_id]
            img = ldcc_load(filename)
            if img is not None:
                return [img],[child_id]
        elif child_id.find('mp4')!=-1:
            files = path_dict[child_id]
            imgs_in_video = []
            ids_in_video = []
            for i,filename in enumerate(files):
                img = ldcc_load(filename)
                if img is not None:
                    imgs_in_video.append(img)
                    ids_in_video.append(child_id.split('.')[0]+'_'+str(i)+'.mp4.ldcc')
            return imgs_in_video, ids_in_video
        return [],[]
    
    elif level=='Parent':
        imgs_in_parent = []
        ids_in_parent = []
        #load and return all jpgs in parent
        for child_id in parent_dict[child_dict[child_id]]:
            if child_id.find('jpg')!=-1:
                filename = path_dict[child_id]
                img = ldcc_load(filename)
                if img is not None:
                    imgs_in_parent.append(img)
                    ids_in_parent.append(child_id)
        return imgs_in_parent, ids_in_parent
    else:
        return [],[]

In [ ]:
#merging kfrm_cap_batch and img_cap_batch together
def img_cap_batch_gen_test(imgs,sens,ids,key,path_dict,id2time_dict):
    if len(imgs)==0:
        return np.array([]),sens,['no_img']*len(sens)
    
    if key.split('.')[1]=='mp4':
        dtype = 'Keyframe'
        doc2time_dict = doc2time(key,path_dict)
    else:
        dtype = 'JPG'
    
    sen_batch = []
    img_batch = []
    img_info_batch = []
    for sen in sens:
        if dtype=='Keyframe':
            img_ids = sen2kfrm(sen,key,doc2time_dict,id2time_dict)
        elif dtype=='JPG':
            img_ids = ids
            
        for i,img_id in enumerate(img_ids):
            if dtype=='Keyframe':
                num = int(img_id.split('_')[1])
                img = imgs[num-1]
                ftype = '.mp4.ldcc'
            elif dtype=='JPG':
                img = imgs[i]
                ftype = '' #it already contains .jpg.ldcc
            sen_batch.append(sen)
            img_batch.append(cv2.resize(img,(299,299)))
            img_info_batch.append([img_id+ftype,(img.shape[0:2])])
    img_batch = np.array(img_batch)        
   
    return img_batch,sen_batch,img_info_batch

def kfrm_cap_batch_gen(kfrms,sens,key,path_dict,id2time_dict):
    doc2time_dict = doc2time(key,path_dict)
    sen_batch = []
    kfrm_batch = []
    kfrm_info_batch = []
    for sen in sens:
        kfrm_ids = sen2kfrm(sen,key,doc2time_dict,id2time_dict)
        for kfrm_id in kfrm_ids:
            num = int(kfrm_id.split('_')[1])
            kfrm = kfrms[num-1]
            sen_batch.append(sen)
            kfrm_batch.append(cv2.resize(kfrm,(299,299)))
            kfrm_info_batch.append([kfrm_id+'.mp4.ldcc',(kfrm.shape[0:2])])
    kfrm_batch = np.array(kfrm_batch)        
    return kfrm_batch,sen_batch,kfrm_info_batch

def img_cap_batch_gen(imgs,sens,ids):
    sen_batch = []
    img_batch = []
    img_info_batch = []
    for sen in sens:
        for i,img in enumerate(imgs):
            sen_batch.append(sen)
            img_batch.append(cv2.resize(img,(299,299)))
            img_info_batch.append([ids[i],(img.shape[0:2])])
    img_batch = np.array(img_batch)        
    return img_batch,sen_batch,img_info_batch

In [ ]:
%%time
print('Creating dictionary and filtering out for grounding...')
filter_out=['pronominal_mention','GeopoliticalEntity','Organization',
            'Location','Money','NumericalValue','Time','URL']
id2men = utils.create_entity_dict(en2men, path_dict, caption_alignment_path=[], filter_out=filter_out)
en_types_filtered=set([])
for id_ in id2men:
    for sen in id2men[id_]:
        for en in id2men[id_][sen]:
            en_type = id2men[id_][sen][en]['type_rdf'].split('#')[1]
            en_types_filtered.update([en_type])
en_types = set([])
for en in en2men:
    type_rdf = en2men[en]['type_rdf']
    en_type = type_rdf.toPython().split('#')[1]
    en_types.update([en_type])
men_types = set([])
for en in en2men:
    for men in en2men[en]['mentions']:
        men_type = en2men[en]['mentions'][men]['mention_type']
        men_types.update([men_type])